# Neural Computation (Autumn 2019)
# Lab 6: Automatic Differentiation (Autograd)

Last week, we gave you a very brief introduction to Pytorch. This week, we will dive more deeply into the Deep Learning framework. By the end of this tutorial, you will know:

- How to create tensors in Pytorch via `torch.Tensor`
- How backpropgation is performed in Pytorch (via `autograd`)
- How to split the dataset into training and validation sets
- How to build a model using `nn.Module` and `nn.Linear` and train that model
- How to evaluate the trained model using the validation set

# Pytorch Tensors

The most basic building block of any Deep Learning library is tensors, which are matrix-like data structures very similar to Numpy's ndarrays with the advantage being that Tensors can also be stored/used on a GPUs to accelerate computing. A scalar has zero dimension, so is called 0D tensor. A vector (i.e., 1D tensor) has one dimension, while a matrix (2D tensor) has two dimension. Note that we introduced tensors in Lab 3 (Advanced Numpy, Tensors & Tensor Operations). If you do not remember what a tensor is, please revisiting Lab 3 again.

In this section, we will show you how to create a tensor using Pytorch. First, we need to `import` the package.

In [0]:
import torch
import numpy as np

You can construct a 4x3 randomly initialised matrix:

In [0]:
x = torch.rand(4,3)
print(x)

Construct a tensor directly from data:

In [0]:
x = torch.tensor([3, 4.5])
print(x)

You can retrieve the size of the tensor using `size` as follows:

In [0]:
print(x.size())

There are many operations that can be performed on tensors. The addition operation can be done as follows:

In [0]:
x = torch.rand(3,4)
y = torch.rand(3,4)
print(x+y)

Or you can use another syntax:

In [0]:
print(torch.add(x, y))

You can also provide an output tensor as argument as follows:

In [0]:
result = torch.empty(3,4)
torch.add(x, y, out=result)
print(result)

You can also convert a Torch Tensor to a Numpy Array using `numpy()` function.

In [0]:
x = torch.ones(5)
print(x)
print(x.dtype)

y = x.numpy()
print(y)
print(y.dtype)

Convert Numpy Array to Torch Tensor using `torch.from_numpy()`. Notice how changing the np array has changed the Torch Tensor automatically.

In [0]:
x = np.ones(3)
y = torch.from_numpy(x)

np.add(x, 1, out=x)
print(x)
print(y)

Tensors can be moved onto any device using the `.to` method. You can check whether a GPU is available using the `torch.cuda.is_available()` method. The `.to()` method sends your tensor to whatever device you specify, including your GPU (referred to as `cpu`) or your GPUs (referred to as `cuda` or `cuda:0`).

In [0]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'  # check whether a GPU is available
y = torch.ones(3,4, device=device)  # directly create a tensor on GPU
x = torch.rand(3,4).to(device)      # or just use .to(device) 
z = x+y
print(z)

As you have seen, `torch.Tensor` is the central class of the Pytorch package. If you now set its attribute `.require_grad` as `True`, it starts to track all operations on it. When you finish your computation you can call `.backward()` and have all the gradients computed automatically. The gradient for this tensor will be accumulated into `.grad` attribute.

The following piece of codes creates a tensor and set `requires_grad=True` to track computation with it.

In [0]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

Perform a tensor operation:

In [0]:
y = x + 2
print(y)

A `Function` is also an important class for autograd implementation in Pytorch. ``Tensor`` and ``Function`` are interconnected and build up an acyclic
graph, that encodes a complete history of computation. Each tensor has
a ``.grad_fn`` attribute that references a ``Function`` that has created
the ``Tensor`` (except for Tensors created by the user - their
`grad_fn is None`). In the example above, you see that `y` has a `grad_fn` since it was created as a result of an operation. We can print the value of that attribute:

In [0]:
print(y.grad_fn)

You can do more operations on `y`:

In [0]:
z = y * y * 2
out = z.mean()

print(z)
print(out)

You can change an existing Tensor's `requires_grad` flag in-place by using `.requires_grad_(...)`. The input flag defaults to `False` if not given. Note that anu operation that mutates a tensor in-place is post-fixed with an `_`. For example, `x.copy_(y)` and `x.t_()` will change `x`.

In [0]:
x = torch.randn(2, 2)
print(x.requires_grad)  # default to False

In [0]:
x.requires_grad_(True)   # change the flag to True
print(x.requires_grad)

In [0]:
y = (x + 1).sum()
print(y.requires_grad)
print(y.grad_fn)

# Gradients

Let's backprop now. Consider the fowllowing piece of codes:

In [0]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out= z.mean()

In [0]:
print("1: ", x)
print("2: ", y)
print("3: ", y.grad_fn)
print("4: ", z)
print("5: ", out)

If you want to compute the derivatives, you can call ``.backward()`` on
a ``Tensor``. If ``Tensor`` is a scalar (i.e. it holds a one element
data), you don’t need to specify any arguments to ``backward()``,
however if it has more elements, you need to specify a ``gradient``
argument that is a tensor of matching shape.

In [0]:
out.backward()

Since `out` contains a single scalar (i.e., mean), `out.backward()` is equivalent to `out.backward(torch.tensor(1.))`.  You can retrieve the gradient of d(out)/dx as follows:

In [0]:
print(x.grad)

You should have got a matrix of ``4.5``. Let’s call the ``out``
*Tensor* “$o$”.
We have that $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.
Therefore,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



Mathematically, if you have a vector valued function $\vec{y}=f(\vec{x})$,
then the gradient of $\vec{y}$ with respect to $\vec{x}$
is a Jacobian matrix:

\begin{align}J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\end{align}

Generally speaking, ``torch.autograd`` is an engine for computing
vector-Jacobian product. That is, given any vector
$v=\left(\begin{array}{cccc} v_{1} & v_{2} & \cdots & v_{m}\end{array}\right)^{T}$,
compute the product $v^{T}\cdot J$. If $v$ happens to be
the gradient of a scalar function $l=g\left(\vec{y}\right)$,
that is,
$v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$,
then by the chain rule, the vector-Jacobian product would be the
gradient of $l$ with respect to $\vec{x}$:

\begin{align}J^{T}\cdot v=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\left(\begin{array}{c}
   \frac{\partial l}{\partial y_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial y_{m}}
   \end{array}\right)=\left(\begin{array}{c}
   \frac{\partial l}{\partial x_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial x_{n}}
   \end{array}\right)\end{align}

(Note that $v^{T}\cdot J$ gives a row vector which can be
treated as a column vector by taking $J^{T}\cdot v$.)

This characteristic of vector-Jacobian product makes it very
convenient to feed external gradients into a model that has
non-scalar output.

# How to define a model in Pytorch?

## An example 

To illustrate this step, let's create some synthetic data. We choose a vector of some points for our feature `x` and create our labels (targets) using the following model $y=a + b \cdot x + \epsilon\cdot  \mathcal{N}$, where `a`, `b` and $\epsilon$ are some constants and $\mathcal{N}$ is noise following a standard normal distribution (mean=0, variance=1).

Let's consider `a=1`, `b=2` and $\epsilon=0.1$. Could you please generate a *column* vector of 100 points for our feature `x` and another *column* vector of the same size for our target `y`? Hint: you can generate samples which follow a standard normal distribution using `np.random.randn(...)`.

In [0]:
# Add you codes here (use x and y as variable names for input and tagets)
np.random.seed(42)
x = ....
y = ....

Now we split the dataset into a trainning set (used for training, about 80% of the original dataset) and a validation set (used for testing, about 20%). We need to shuffle the array of indices:

In [0]:
indx = np.arange(100)  # indices for all data points in x
print("Before shuffle: \n", indx)

np.random.shuffle(indx)
print("After shuffle: \n", indx)

Now split the indices into two sets:

In [0]:
train_indx = ....  # first 80% for training
val_indx = ....    # remaining 20% for validation

Split the dataset into training and validation sets:

In [0]:
# Generate inputs and targets for training step
x_train, y_train = x[train_indx], y[train_indx]
print(x_train.size, y_train.size)

# Please generate inputs and targets for validation step
x_val, y_val = ....

We need to convert these Numpy's ndarray into Torch tensors of type `float`:

In [0]:
x_train_tensor = ....
y_train_tensor = ....

x_val_tensor = ....
y_val_tensor = ....

We can also plot the points using the `scatter` method.

In [0]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(nrows = 1, ncols = 2)
axs[0].scatter(x_train, y_train)  # plot the training dataset
axs[1].scatter(x_val, y_val)      # plot the validation dataset

## A simple linear gression model

Let's difine a simple linear regression model to learn the values for two parameters `a` and `b` in the model above.

As you might see last week, a model can be constructed in Pytorch using the `torch.nn` class. To explicitly define the model, you need to implement (at least) the following methods:

- `__init__(self)`, which defines the components that make up the model. Here, you are not limited to defining parameters and other models (or layers in neural networks) as our model's attributes (see more about this later).
- `forward(self, x)`, which performs the actual computation, that is, it ouputs a prediction, given the input `x`. You need not call the `forward(x)` method, and should call the whole model itself to perform a forward pass and output predictions.

Out first model will look like this:

In [0]:
import torch.nn as nn

class FirstModel(nn.Module):

    def __init__(self):
        super().__init__()

        # add your codes here (hint: last week's lab)
        self.a = ....
        self.b = ....

    def forward(self, x):
        return ....

In the `__init__` method, we define two parameters, `a` and `b`, using the `Parameter()` class. By doing this, you can invoke the `parameters()` method of our model to retrieve an iterator over all model's parameters, that we can feed our optimiser. Moreover, we can get the current values for all parameters using the model's `state_dict()` method.

In the following, we will use stochastic gradient descent, i.e., the `SGD` method from `torch.optim` package, which takes two arguments: the list of model's parameters (`model.parameters()`) and the learning rate `lr`.

In [0]:
import torch.optim as optim
torch.manual_seed(42)

# Now we can create a model
model = FirstModel().to(device)

# we can also inspect its parameters
print("Before training: \n", model.state_dict())

In [0]:
# set learning rate
lr = 1e-1

# set number of epoches, i.e., number of times we iterate through the training set
epoches = 100

# We use mean square error (MSELoss)
loss_fn = nn.MSELoss()

# PLEASE use stochastic gradient descent (SGD) to update a and b
optimiser = ....

for epoch in range(epoches):
    model.train()             # set the model to training mode 
    optimiser.zero_grad()     # avoid accumulating gradients
    y_pred = ....             # PLEASE calculate model's output
    loss = ....               # PLEASE calculate the MSE loss
    loss.backward()           # calculate gradients
    optimiser.step()          # update model's params

print("After training: \n", model.state_dict())

When setting the number of epoches sufficiently large, you should be able to obtain some values for parameters `a` and `b` which are really close to their ground-truth values of 1 and 2. 

**Please change the number of epoches (`epoches`) and the learning rate (`lr`) to see how the derived values for the model's parameters have changed.**

## Nested model

In our model above, we manually created two parameters to perform a linear regression. We can also use Pytorch's `Linear` model as an attribute to our model, which results in a nested model. 

To do this, we need to change the `__int__` method, where we create an attribute that contains our nested `Linear` model. Also in the `forward()`, we will simply call the nested model itself to perform the forward pass.

Our new model will look like:


In [0]:
class NewModel(nn.Module):

    def __init__(self):
        super().__init__()
        # a simple linear layer with an input and an output
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return ....

We can now call inspect the model's parameters by calling `parameters()`:

In [0]:
new_model = NewModel().to(device)
print(list(new_model.parameters()))

You should see that the model has two parameters. The first one is the **weight** `b`, while the other one is the **bias** `a` in our linear model defined at the beginning (recall that $y=a + bx + \epsilon \cdot \mathcal{N}$). You can see this clearly when using the `state_dict()` method.

In [0]:
print(new_model.state_dict())

In [0]:
# print the model
print(new_model)  

Because we use SGD as our optimiser, so we need mini-batches (i.e., small subsets of our dataset) to feed the model. In Pytorch, we can use the `DataLoader` class to do this. We simply need to tell it which dataset (of `TensorDataset`) to use, the size of the mini-batch (or simply batch size). The loader is an iterator-like, which can loop over the dataset and fetch a adifferent mini-batch every time.

In [0]:
from torch.utils.data import TensorDataset, DataLoader

train_data = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)

Let's see how we train the model now.

In [0]:
# PLEASE construct the model here
new_model = ....

print("Before training: \n", new_model.state_dict())
lr = 1e-1
epoches = 100

# We use mean square error (MSELoss)
loss_fn = ....

# PLEASE use stochastic gradient descent (SGD) to update a and b
optimiser = 

In [0]:
for epoch in range(epoches):
    new_model.train() 
    for x_batch, y_batch in train_loader:
        # send tensors to device (cpu/cuda)
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        # add your codes here
        ....

print("After training: \n", new_model.state_dict())

So far, we have defined an optimiser, a loss function and a (nested) model. We realise that if we use the codes above, we have to modify it whenever we'd like to use a different optimiser or a different loss function. You might think of making our codes more generic. How about writing a function that takes three arguments (optimiser, loss function and a model) and perform the training step. You will see below how to write a function in Python which returns another function.

In [0]:
def generic_code(model, loss_fn, optimiser):

    # define a function inside another function
    def train_step(x_batch, y_batch):
        optimiser.zero_grad()
        y_pred = ....           # forward pass
        loss = ....             # calculate loss value
        loss....                # autograd
        optimiser....             # update parameters  
        return loss.item()             # return the loss

    # return the newly defined function
    return train_step                # return a function

You can now perform the training process as usual:

In [0]:
new_model = NewModel()
print(new_model.state_dict())

lr = 1e-1
epoches = 100

# We use mean square error (MSELoss)
loss_fn = nn.MSELoss()

# We also use stochastic gradient descent (SGD) to update a and b
optimiser = optim.SGD(new_model.parameters(), lr=lr)

train_step = generic_code(new_model, loss_fn, optimiser)
# What is the type of train_step?

# list to record the loss over training course
losses = list()

for epoch in range(epoches):
    for x_batch, y_batch in train_loader:
        # calculate the loss and add it to the loss list
        # add your codes here

print(new_model.state_dict())

You can now plot the loss value over time as follows:

In [0]:
plt.plot(range(len(losses)), losses, label="Training loss")
plt.xlabel("Training iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

## Evaluation

We have not used the validation set that we created at the beginning of the tutorial. We have split the dataset of 100 points into a training set (80%) and a validation set (20%). We now change our model to include the evaluation phase, that is, computing the validation loss. Intuitively speaking, since we do not use the validation set when training the model, so the validation set is actually *unseen* to our model, and turns out to be a good choice to inspect how well our model performs on unseen data (i.e., making predictions). 





We first need to create a DataLoader (`batch_size=20`) for the validation set.

In [0]:
val_dataset = ....
val_loader = ....

Our training loop should look like this. Note that `with torch.no_grad()` helps you stop autograd from tracking history on Tensors with `requires_grad=True`. All you need to do is wrapping the code block in `with torch.no_grad():`. See the following piece of codes for more detail.

In [0]:
new_model = NewModel().to(device)
print(new_model.state_dict())

lr = 1e-1
epoches = 20
loss_fn = nn.MSELoss(reduction='mean')
optimiser = optim.SGD(new_model.parameters(), lr=lr)

losses = list()
val_losses = list()
train_step = generic_code(new_model, loss_fn, optimiser)

for epoch in range(epoches):
    new_model.train()
    for x_batch, y_batch in train_loader:
        # Calculate the loss (using train_step function) 
        # and add it to the list called losses
        ....

        with torch.no_grad():       # fix the model's params
            for x_val, y_val in val_loader:
                new_model.eval()    # set model to evaluation mode
                y_pred = ....       # model's output for an input of x_val
                val_loss = ....     # validation loss
                val_losses.append(....)

print(new_model.state_dict())
plt.plot(range(len(losses)), losses,'r', label="Training loss")
plt.plot(range(len(val_losses)), val_losses,'g', label="Validation loss")
plt.xlabel("Training iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

Given the learning rate in the range of `np.linspace(0.01, 0.1, 50)`. Could you plot a graph showing how the validation loss changes according to different values of the learning rate? Set the number of epoches to a sufficiently small value for a reasonable running time. For each value of the learning rate, you might calculate the **average validation loss** or the **minimum validation loss**.

In [0]:
# add your codes here
